<a href="https://colab.research.google.com/github/jish1398/NLP-intern/blob/master/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data.json to data.json


In [2]:
!pip install spacy==2.1.3

In [3]:
import spacy
import json
import logging
import random
print(spacy.__version__)

2.1.3


In [0]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r',encoding="UTF-8") as f:
            lines = f.readlines()

        for line in lines:

            data = json.loads(line)
            text = data['content']
            entities = []
            
            if(type(data['annotation']) != type(None)):
            
                for annotation in data['annotation']:
                    
                    point = annotation['points'][0]
                    labels = annotation['label']
                    
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        
                        entities.append((point['start'], point['end'] + 1 ,label))


                training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [0]:
data = convert_dataturks_to_spacy("data.json")

In [0]:
td1=data[:26]
td2=data[31:173]
td3=data[177:249]
td4=data[252:300]


train_data = td1+td2+td3


In [0]:
def train_spacy(train_data):
    TRAIN_DATA = train_data
    nlp = spacy.blank('en')  
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

   
    for text, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  
        optimizer = nlp.begin_training()
        for itn in range(100):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.05,  
                    sgd=optimizer,  
                    losses=losses)
            print(losses)
    nlp.to_disk('sample_work_model_300_drop_0.05')

In [18]:
train_spacy(train_data)

Starting iteration 0
{'ner': 52258.25662486403}
Starting iteration 1
{'ner': 49513.929914883694}
Starting iteration 2
{'ner': 52611.097175604846}
Starting iteration 3
{'ner': 39956.04617427384}
Starting iteration 4
{'ner': 33214.202342177465}
Starting iteration 5
{'ner': 42044.17877005543}
Starting iteration 6
{'ner': 34509.18991526442}
Starting iteration 7
{'ner': 40459.78186563896}
Starting iteration 8
{'ner': 38205.90117616678}
Starting iteration 9
{'ner': 34023.440170355985}
Starting iteration 10
{'ner': 30823.740223695997}
Starting iteration 11
{'ner': 25231.223113515196}
Starting iteration 12
{'ner': 23437.822796061937}
Starting iteration 13
{'ner': 24318.909738822935}
Starting iteration 14
{'ner': 17794.59076704312}
Starting iteration 15
{'ner': 16299.870720423714}
Starting iteration 16
{'ner': 19279.544595461197}
Starting iteration 17
{'ner': 19025.193689126325}
Starting iteration 18
{'ner': 15449.998673000848}
Starting iteration 19
{'ner': 17657.586654244496}
Starting iteratio

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6128449890667153985, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4404198431831190453
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8290177503737042560
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11274954343
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6690364833259330222
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
nlp = spacy.load('sample_work_model_300_drop_0.05')

In [0]:
test_data = data[290:]
doc = nlp(test_data[3][0])

In [37]:
print(len(data))

300


In [0]:
def show_docs(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+" --> "+ ent.label_)

In [39]:
show_docs(doc)

Chief Operating Officer --> Role
Chief Operating Officer --> Role
At Will --> At_will_not
annual salary of $175,000 --> Base_Salary
purchase shares of Common Stock --> Stock_options
Chief Operating Officer --> Role


In [27]:
print(test_data[0])

('f8k081710ex10i_ctd.htm\n\n\nExhibit 10.1\n\n\n\n\n\nEMPLOYMENT AGREEMENT\n\n\n\n\n\nTHIS EMPLOYMENT AGREEMENT (the “Agreement”) is effective as of February 15, 2010, (the “Effective Date”) by and between NANOSONIC PRODUCTS, INC., a Florida corporation (the “Company”), and JEFFREY L. TATE, Ph.D. (the “Employee”).\n\n\n\n\n\nRECITALS:\n\n\n\n\n\nThis Agreement is intended to provide for the employment of Employee by the Company from and after the date hereof, all on the terms and conditions herein set forth.\n\n\n\n\n\nNOW, THEREFORE, for and in consideration of the premises and the mutual covenants and agreements herein contained, and for other valuable consideration, the receipt and sufficiency of which are hereby acknowledged, the parties hereby agree as follows:\n\n\n\n\n\n\n\n1.\n\n\nEmployment.\n\n\n\n\n\n\n\n\n\n\n\n\n1.1\n\n\nSubject to Section 3 below, the Company hereby employs Employee for a two- (2) year term beginning on February 15, 2010, and ending February 14, 2012 (the

In [40]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

    100% |████████████████████████████████| 993kB 20.0MB/s 


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       
drive = GoogleDrive(gauth)

In [45]:
nlp.save('spacy_temp.h5')
model_file = drive.CreateFile({'title' : 'spacy_temp.h5'})

AttributeError: ignored

In [46]:
!pwd

/content


In [47]:
!ls


 adc.json	 'data (3).json'   sample_work_model_300_drop_0.05
'data (1).json'   data.json
'data (2).json'   sample_data


In [48]:
from google.colab import files
files.download('sample_work_model_300_drop_0.05')

MessageError: ignored